In [22]:
from models.resnet import *
from models.vgg import *
from models.mobilenetv2 import qmobilenetv2
from models.mobilenet import qmobilenet
from quantize_utils import *
from utils import *
import conf
dataloader = cifar100DataLoader(train=True, shuffle=True, batch_size=128, normalized=True)
# dataloader = cifar10DataLoader(train=True, normalized=False, batch_size=128, shuffle=True)
sampels, label = iter(dataloader).next()

In [14]:
# dataloader = cifar100DataLoader(train=True, shuffle=True, batch_size=128, normalized=True)
dataloader = cifar10DataLoader(train=True, normalized=False, batch_size=128, shuffle=True)
sampels, label = iter(dataloader).next()
model = getCifar10Model('vgg11', pretrained=True)
model.eval()
model.cpu()
sampels = sampels.to('cpu')
IR = [sampels]
idx = []
with torch.no_grad():
    for i in range(29):
        IR.append(model.features[i](IR[-1]))
        if type(model.features[i]) in [torch.nn.Conv2d, QConv2d]:
            idx.append(i)
IR = [IR[i] for i in idx]

In [17]:
model = vgg19_bn()
load_qnet(model, conf.vgg19_path)
model.eval()
model.cpu()
sampels = sampels.to('cpu')
IR = [sampels]
idx = []
with torch.no_grad():
    for i in range(53):
        IR.append(model.features[i](IR[-1]))
        if type(model.features[i]) == torch.nn.Conv2d:
            idx.append(i)
IR = [IR[i] for i in idx]

In [23]:
res18 = qresnet18()
load_qnet(res18, conf.resnet18_path)
# mixed_quantize_with_partition(res18, [2] * (lenQmodel(res18)-1), split=lenQmodel(res18)-1, a_bit=8)
res18.eval()
res18.cpu()
sampels = sampels.to('cpu')
IR = []
IR.append(res18.conv1(sampels))
IR.append(res18.conv2_x[0](IR[-1]))
IR.append(res18.conv2_x[1](IR[-1]))
IR.append(res18.conv3_x[0](IR[-1]))
IR.append(res18.conv3_x[1](IR[-1]))
IR.append(res18.conv4_x[0](IR[-1]))
IR.append(res18.conv4_x[1](IR[-1]))
IR.append(res18.conv5_x[0](IR[-1]))
IR.append(res18.conv5_x[1](IR[-1]))
x = res18.avg_pool(IR[-1])
y = res18.fc(x.view(x.size(0), -1))
_, pred = y.topk(5, 1, largest=True, sorted=True)
label = label.view(label.size(0), -1).expand_as(pred)
correct = pred.eq(label).float()
print(correct[:, :1].sum() / y.shape[0])
IR.insert(0, res18.conv1[0](sampels))

tensor(1.)


In [ ]:
sampels = sampels.to('cpu')
IR_e = []
for i in range(lenQmodel(res18, [BasicBlock])):
    next_block_idx = bw_split(res18, [BasicBlock, BottleNeck], i)
    res18(sampels)
    IR_e.append(extract_IR(res18))

In [ ]:
res50 = qresnet50()
load_qnet(res50, conf.resnet50_path)
# test(res50, dataloader, 'cuda:1')
res50.eval()
res50.cpu()
sampels = sampels.to('cpu')
IR = []
IR.append(res50.conv1(sampels))
IR.append(res50.conv2_x[0](IR[-1]))
IR.append(res50.conv2_x[1](IR[-1]))
IR.append(res50.conv2_x[2](IR[-1]))
IR.append(res50.conv3_x[0](IR[-1]))
IR.append(res50.conv3_x[1](IR[-1]))
IR.append(res50.conv3_x[2](IR[-1]))
IR.append(res50.conv3_x[3](IR[-1]))
IR.append(res50.conv4_x[0](IR[-1]))
IR.append(res50.conv4_x[1](IR[-1]))
IR.append(res50.conv4_x[2](IR[-1]))
IR.append(res50.conv4_x[3](IR[-1]))
IR.append(res50.conv4_x[4](IR[-1]))
IR.append(res50.conv4_x[5](IR[-1]))
IR.append(res50.conv5_x[0](IR[-1]))
IR.append(res50.conv5_x[1](IR[-1]))
IR.append(res50.conv5_x[2](IR[-1]))
x = res50.avg_pool(IR[-1])
y = res50.fc(x.view(x.size(0), -1))
_, pred = y.topk(5, 1, largest=True, sorted=True)
label = label.view(label.size(0), -1).expand_as(pred)
correct = pred.eq(label).float()
print(correct[:, :1].sum() / y.shape[0])
IR.insert(0, res50.conv1[0](sampels))

In [ ]:
mov = qmobilenet()
load_qnet(mov, conf.mobilenet_path)
# mixed_quantize_with_partition(mov, [6] * (lenQmodel(mov)-1), split=lenQmodel(mov)-1, a_bit=8)
mov.eval()
mov.cpu()
sampels = sampels.to('cpu')
IR = []
IR.append(mov.stem[0](sampels))
IR.append(mov.stem[1](IR[-1]))
IR.append(mov.conv1[0](IR[-1]))
IR.append(mov.conv1[1](IR[-1]))
IR.append(mov.conv2[0](IR[-1]))
IR.append(mov.conv2[1](IR[-1]))
IR.append(mov.conv3[0](IR[-1]))
IR.append(mov.conv3[1](IR[-1]))
IR.append(mov.conv3[2](IR[-1]))
IR.append(mov.conv3[3](IR[-1]))
IR.append(mov.conv3[4](IR[-1]))
IR.append(mov.conv3[5](IR[-1]))
IR.append(mov.conv4[0](IR[-1]))
IR.append(mov.conv4[1](IR[-1]))

x = mov.avg(IR[-1])
x = x.view(x.size(0), -1)
y = mov.fc(x)
_, pred = y.topk(5, 1, largest=True, sorted=True)
label = label.view(label.size(0), -1).expand_as(pred)
correct = pred.eq(label).float()
print(correct[:, :1].sum() / y.shape[0])
IR.insert(0, mov.stem[0].conv(sampels))

In [ ]:
mov2 = qmobilenetv2()
load_qnet(mov2, conf.mobilenetv2_path)
mixed_quantize_with_partition(mov2, [8] * (lenQmodel(mov2)-1), split=53, a_bit=-1)
mov2.eval()
mov2.cpu()
sampels = sampels.to('cpu')
IR = []
IR.append(mov2.pre(sampels))
IR.append(mov2.stage1(IR[-1]))
IR.append(mov2.stage2[0](IR[-1]))
IR.append(mov2.stage2[1](IR[-1]))
IR.append(mov2.stage3[0](IR[-1]))
IR.append(mov2.stage3[1](IR[-1]))
IR.append(mov2.stage3[2](IR[-1]))
IR.append(mov2.stage4[0](IR[-1]))
IR.append(mov2.stage4[1](IR[-1]))
IR.append(mov2.stage4[2](IR[-1]))
IR.append(mov2.stage4[3](IR[-1]))
IR.append(mov2.stage5[0](IR[-1]))
IR.append(mov2.stage5[1](IR[-1]))
IR.append(mov2.stage5[2](IR[-1]))
IR.append(mov2.stage6[0](IR[-1]))
IR.append(mov2.stage6[1](IR[-1]))
IR.append(mov2.stage6[2](IR[-1]))
IR.append(mov2.stage7(IR[-1]))
IR.append(mov2.conv1(IR[-1]))

x = torch.nn.functional.adaptive_avg_pool2d(IR[-1], 1)
x = mov2.conv2(x)
y = x.view(x.size(0), -1)
_, pred = y.topk(5, 1, largest=True, sorted=True)
label = label.view(label.size(0), -1).expand_as(pred)
correct = pred.eq(label).float()
print(correct[:, :1].sum() / y.shape[0])
IR.insert(0, mov2.pre[0](sampels))

In [24]:
# model = mov
with torch.no_grad():
    for i, ir in enumerate(IR):
        ir_images = ir2images(ir, (32, 32))
        sampels = sampels.to('cuda:1')
        # print(1 / maxSSIM(ir_images, sampels.to('cuda:1')))
        # print(minKL(ir_images, sampels.to('cuda:1'), res18.to('cuda:1')))
        print(i, ',', minMSE(ir_images, sampels, device='cuda').item())

0 , 0.22897884249687195
1 , 0.8808073997497559
2 , 0.8732382655143738
3 , 0.8745739459991455
4 , 0.8875979781150818
5 , 0.8894134759902954
6 , 0.9964187145233154
7 , 1.0245037078857422
8 , 1.0736452341079712
9 , 1.1709766387939453
